**ELABORATION DE LA BASE DE CONTRAINTES TEMPORELLES ORDONNEE**

Entrée du programme : liste de paires, (Evenement, Timestamp)

Representation finale de la base : pour chaque paire d'evenement, une liste de noeuds


Chaque noeud = intervalle, liste de fils, liste de pères

**ALGORITHME HCDA**

In [0]:
import os as os

Methodes intermédiaires

In [0]:
#-------------------------------------------------------------------------------------------------------------------------------
#ATTENTION !!!! Toujours ajouter les elements dans une liste avec append() => garde l'ordre (MEME DANS add_as_...)
#-------------------------------------------------------------------------------------------------------------------------------
#structure des elements d'un tableau pour une paire d'evenement
#C = liste de Node (voir classe au-dessus)
#
#forme de D = [ [C1, C2, C3...], [C1, C2, C3...], [C1, C2, C3...]...]
#-------------------------------------------------------------------------------------------------------------------------------
# fonctions à implémenter (numérotées)

#plus de num 1, c'etait pas utile :)

In [0]:
# num 2 on a en entrée Chronique, et une liste- on veut savoir si dans la liste il y a une chronique dont l'ID est plus général que la chronique en parametre [1,6] et [1,7] |Boolean
def has_more_general(L, C):
  found = False
  k=len(L)
  while not(found) and k>0:
    k = k-1
    N = L[k]
    if N.id[0] <= C.id[0] and N.id[1] >= C.id[1] :
      found = True
  return found

In [0]:
# num 3
#enlever egalité sur le if #NON
#recherche en partant de la fin... pas vraiment de raison - Inverse de num2
def has_stricter(L, C):
  found = False
  k=len(L)
  while not(found) and k>0:
    k = k-1
    N = L[k]
    if N.id[0] >= C.id[0] and N.id[1] <= C.id[1] :
      found = True
  return found

In [0]:
# num 4

# V3 : on calcule en supposant que tous les évènements sont rangés par ordre 
#      d'apparition chronologique dans S

# attention au niveau du calcul de fréquence --> la frequence calculée est trop élevée 
#reconnaissance de contraintes (partie de chronique) !! Attention CRS=chronicle. 

# On veut compter le nombre d'occurence de C dans la séquence S (C=Node et S=séquences des évenements).  
#ev1 et ev2 => A C et S=[('A',3),('B',4),('C',2),('B',3)]

def count_crs(ev1, ev2, C, S):
  x = C.id[0]
  y = C.id[1]
  t = len(S)
  #Mise en place d'une liste de tuple
  listTATB=[]

  #priorité à x sur -x :
  #pour un intervalle -3,5 on cherche a obtenir list_i = 0,1,-1,2,-2,3,-3,4,5
  #list_i = parkour_i_S(x, y)
  #size = y-x+1

  used = [] #liste des indices dans S des evenements utilisés
  frequence = 0
  
  for i in range(0, t):
    if(S[i][0]==ev1): #premier événement trouvé

      #alors on cherche 1 occurrence qu'il pourrait former:
      #parcours pour trouver les ev2

      #la plus proche en terme d'écart absolu => parcours autour de l'évènement en question
      #a chaque boucle, on regarde l'évement à i+l puis à i-l
      #MAIS     les parcours dans les "negatifs" et les "positifs" 
      #         s'arretent individuellement si ils deviennent hors champs
      #         c'est à dire qu'il ne fourniront pas d'occurence
      l=1
      found = False

      upper_finished = False
      lower_finished = False

      #on gardera l'occurence la plus proche en terme d'écart
      best_occ = -1 #range in S for ev2

      upper_found = False
      upper_gap = y+1
      upper_rg = -1

      lower_found = False
      lower_gap = x-1
      lower_rg = -1

      #begin while-----------------------------------------------------
      #stop : (les deux cotes sont arretes) ou (l'element le plus proche est trouvé)
      while(not found and not(upper_finished and lower_finished)):
        
        #------------ test upper : i+l ----------------
        rg = i+l
        if(not upper_finished): 
          
          if(rg<t and rg >= 0): #RAISON 1 : dans la séquence !!!!!!!!!!!! ATTENTION !!!!!!!!!!!!!

            gap_i_rg = S[rg][1] - S[i][1]
            if (gap_i_rg<=y): #RAISON 2 : ecart dans l'intervalle
              
              if(not(lower_found and gap_i_rg > abs(lower_gap))): #RAISON 3 : the potential upper occurence could be better than lower
                
                #validation occ
                if(S[rg][0]==ev2 and (rg not in used)):
                  upper_found = True
                  upper_gap = gap_i_rg
                  upper_rg = rg
                  upper_finished = True

              else:
                #RAISON 3 : the lower occurence is the best !
                upper_finished = True
                
            else:
              #RAISON 2 : ecart pas dans l'intervalle !
              upper_finished = True
            
          else:
            #RAISON 1 : pas dans la séquence !
            upper_finished = True
        

        #--------------- test lower : i-l --------------
        rg = i-l
        if(not lower_finished): 
          
          if(rg<t and rg>=0): #RAISON 1 : dans la séquence !!!!!!!!!!!! ATTENTION !!!!!!!!!!!!!

            gap_i_rg = S[rg][1] - S[i][1]
            if (gap_i_rg>=x): #RAISON 2 : ecart dans l'intervalle
              
              if(not(upper_found and abs(gap_i_rg) >= upper_gap)): #RAISON 3 : the potential lower occurence could be better than upper
                
                #validation occ
                if(S[rg][0]==ev2 and (rg not in used)):
                  lower_found = True
                  lower_gap = gap_i_rg
                  lower_rg = rg
                  lower_finished = True

              else:
                #RAISON 3 : the upper occurence is the best !
                lower_finished = True
                
            else:
              #RAISON 2 : ecart pas dans l'intervalle !
              lower_finished = True
            
          else:
            #RAISON 1 : pas dans la séquence !
            lower_finished = True

        #----------- test if found, who is it -----------
        if(lower_found):

          found = True
          if(upper_found):
            if(upper_gap <= abs(lower_gap)):
              best_occ = upper_rg
            else:
              best_occ = lower_rg
          else:
            best_occ = lower_rg

        elif(upper_found):
          found = True
          best_occ = upper_rg

        l=l+1

      #end while----------------------------------------------------

      #trouvée
      if(found):
        frequence = frequence + 1

        #Ajout des élements [TA, TB] avec TA =i et TB=best_occ => ranges in S
        listTATB.append([i,best_occ])

        used.append(best_occ)
     
  return [frequence,listTATB]


In [0]:
# num 5
#supprimer les plus stricts (utile!)
#Parcourir L et supprimer les nodes de L qui ont un intervalle, c'est à dire un ID, plus strict contenu dans l'ID de C
#Puis ajouter C dans la liste. 

def add_as_minimal(C, L, f,frequences,temporary_list, List_TATB):

  k=len(L)

  while k>0:
    k = k-1
    N = L[k]
    if N.id[0] <= C.id[0] and N.id[1] >= C.id[1] :
      L.pop(k)
      List_TATB.pop(k)
      frequences.pop(k)

  L.append(C)
  frequences.append(f)
  List_TATB.append(temporary_list)
  

In [0]:
# num 6

#L'inverse de Num 5. 
def add_as_maximal(C, L):
  k=len(L)

  while k>0:
    k = k-1
    N = L[k]
    if N.id[0] >= C.id[0] and N.id[1] <= C.id[1] :
      L.pop(k)

  L.append(C)



In [0]:
# num 7
# D : constraint database from ccdc
def generate_children(C, tree):
  children = []
  for id_child in C.sons:
    found = False
    i = 0
    k = len(tree)
    while i < k and not(found):
      if tree[i].id == id_child:
        children.append(tree[i])
        
      i = i+1
  return children

In [0]:
# A DEFINIR
# contraintes ajoutées par l'utilisateur
#si event correspond à un evenement qu'on ne veut pas event=['A', 'B'] et C c'est un noeud. 
def satisfies_monotonic_constraints(event, C):
  return True

In [0]:
# num 8
def never_processed(id_processed, child):
  found = False
  k = len(id_processed)
  while not(found) and k>0:
    k = k-1
    if id_processed[k] == child.id :
      found = True
  return not(found)

In [0]:
# num 9
#attention a pas ajouter une chronique déjà candidate (ok)
def add_candidate(L, E):
  exists = False
  for ele in L:
    if ele == E:
      exists = True
  
  if not(exists):
    L.append(E)
 
 

In [0]:
def transform_ntoc(frequents_nodes, even,list_freq, list_tatb):
  Frequents = []
  i = 0

  #Pour chaque élement de frequents_nodes, on récupere l'intervalle associé à even (pas les fils ni les peres) et on les range dans Frequents. 
  for k in frequents_nodes:
    Frequents.append([k.id,even,list_freq[i],list_tatb[i]])
    i = i+1
  return Frequents

In [0]:
# D : constraint database from ccdc
# S : sequence
# fth : fréquence minimale de signification

#HCDA appliqué sur un seul sub_tree de la CCDC, c'est a dire un arbre associé à une seule paire de d'évènement

def HCDA_sub(even, tree, S, fth):

  Frequents = [] # minimal (!) discovered chronicles
  Non_Frequents = [] # maximal (!) 
  id_processed = []
  frequences = []
  List_TATB = []

  if(tree != []):
    Candidates = [tree[0]] #root

    while Candidates != [] :

      C = Candidates[0] # C : current candidate

      if not has_more_general(Non_Frequents, C):
        #if has_stricter(Frequents, C): #pk deja ???? je mettrais plutot not 
        
        #if not has_stricter(Frequents, C):
        #  on a pas assez d'info donc on teste sa fréquence => il devient Frequents ou Non-Frequents
        #  Dans le cas non frequent : on n'analyse pas ses fils
        #else: il est frequent mais pas le minimal donc on l'ajoute pas dans frequents

        is_frequent = True

        if not has_stricter(Frequents, C):
          Temp = count_crs(even[0], even[1], C, S)
          fcrs=Temp[0]
          temporary_list =Temp[1]
          #print("### test 1###")
          #print(Temp)
          #print("*************")

          if fcrs < fth:
            add_as_maximal(C, Non_Frequents) #maximal => supprimer les plus stricts (utile!)
            is_frequent = False
          else:
            add_as_minimal(C, Frequents,fcrs, frequences,temporary_list, List_TATB) #minimal => supprimer les plus généraux
            #frequences.append(fcrs)
            
            #ajouter ici .append
            # add_as_minimal(C, frequences)

            #print("### test 2###")
            #print(frequences)
            #print("********3*****")
            #print(Temp)
            #print("*************")

        if is_frequent:
          Children = generate_children(C, tree)

          for child in Children:

            if satisfies_monotonic_constraints(even, child) and never_processed(id_processed, child):
              add_candidate(Candidates, child)
      
      #mettre C en "processed"
      id_processed.append(C.id)
      Candidates.pop(0)
      #print(frequences)
      
      #print("### test 45###")
      #print(frequences)
      #print("*************")
      #print(Frequents)

  return [Frequents,frequences,List_TATB]

Algorithme principal

In [0]:
  #HCDA appliqué sur la liste E 
#C = base crée par CCDC

def HCDA(S, fth, limite_ecarts,C):
    Assembled_Chronicle = []
    for ele in C:
      #recuperation a partir de l'élement, even correspond à ele<(0] et tree=ele[1]
      even = ele[0]
      tree = ele[1] #sans l'information "['A', 'B']"

      result_HCDA = HCDA_sub(even, tree, S, fth)
      frequents_nodes = result_HCDA[0]

      Temp_freq=result_HCDA[1]
      Temp_list_TATB=result_HCDA[2]

      #print("### test 4 ###")
      #print(frequents_nodes)
      #print("*********5****")
      #print(Temp_freq)
      #print("### fin test ###")

      frequents_chrns = transform_ntoc(frequents_nodes, even,Temp_freq, Temp_list_TATB)

      for c in frequents_chrns:
        Assembled_Chronicle.append(c)
    
    return Assembled_Chronicle

In [0]:
from tabulate import tabulate

def print_HCDA(H):
  headers = ["time constraint", "events", "frequence", "occurences"]
  print(tabulate(H, headers=headers))

**TEST SUR LE DATASET SALADES**

In [0]:
def sequence(filepath):
  data = []
  temp = []
  #filepath = 'datasetP/01-1-activityAnnotation_arranged.txt'
  with open(filepath) as fp:
    line = fp.readline()
    while line:
       # print("Line {}: {}".format(cnt, line.strip()))
        line = fp.readline()
        list_line = line.split()
        #print(list_line)
        if list_line != []:
            data.append((list_line[1], int(list_line[0])))
            #temp.append(list_line)

  return data

In [0]:
def get_sequences(path):
   seq = []
   for name in get_all_files(path):
      seq.append(sequence(name))
   return seq

In [0]:
def get_all_files(path):
  os.chdir(path)
  #traces = os.system('ls')
  #type(traces)
  all_file_names = dirlist = os.listdir()
  return all_file_names

In [0]:
# retourne le timestamp maximum de tout les datas et la precision (ecart min entre les timestamps)

def exploration(datasets):
  maxis = []
  all_gaps = []
  for i in datasets:
    tt = []
    for j in i:
      tt.append(j[1])

    gaps = [tt[i+1]-tt[i] for i in range(len(tt)-1)]
    all_gaps.append(gaps)
    maxis.append(max(tt))

    minis = []
    max_gaps = []
    for i in all_gaps:
      max_gaps.append(max(i))
      mi = min(i)
      if mi != 0:   
        minis.append(min(i))
  
  max_gap = max(max_gaps)
  precision = min(minis)
  max_timestamps = max(maxis)

  print("timestamp le plus grand : {}".format(max_timestamps))
  print("ecart min entre timestamps : {}".format(precision))
  print("ecart max entre timestamps : {}".format(max_gap))

  return max_timestamps


In [0]:
def modif_timestamps(data,facteur):
  all_data = []
  for dataset in data:
    liste = []
    for pair in dataset:
      #print(pair)
      temp =list(pair)
      val = temp[1]
      #print(val)
      #new_val = val-150000
      new_val = int(val*facteur)
      #print(new_val)
      new_pair = (temp[0],new_val)
      #print(new_pair)
      #tuple(temp)
      liste.append(new_pair)
    all_data.append(liste)
  return all_data

In [0]:
def decal_concat(data):

#  if modif == True :
 #   data = modif_timestamps(data,facteur)

  decal = exploration(data)

  # recupere le dernier timestamp de chaque sequence 
  decalages = []
  for dataset in data:
    decalages.append(dataset[-1][1])

  all_data = []
  #la premiere sequence reste identique
  all_data.append(data[0])
  #pour chaque seq suivante, on décale les valeurs en fct du max de celle d'avant
  i = 1
  for seq in data[1:]:
    temp_seq = []
    for pair in seq:
      temp = list(pair)    
      new_val = temp[1] + decal + sum(decalages[0:i])
      new_pair = (temp[0],new_val)
      temp_seq.append(new_pair)
    i += 1
    all_data.append(temp_seq)
  
  flat_list = [item for sublist in all_data for item in sublist]

  return flat_list

In [0]:
def decalage(data,modif,facteur):

  if modif == True :
    data = modif_timestamps(data,facteur)

  # recupere le dernier timestamp de chaque sequence 
  decalages = []
  for dataset in data:
    decalages.append(dataset[-1][1])

  all_data = []
  #la premiere sequence reste identique
  all_data.append(data[0])
  #pour chaque seq suivante, on décale les valeurs en fct du max de celle d'avant
  i = 1
  for seq in data[1:]:
    temp_seq = []
    for pair in seq:
      temp = list(pair)    
      new_val = temp[1] + sum(decalages[0:i])
      new_pair = (temp[0],new_val)
      temp_seq.append(new_pair)
    i += 1
    all_data.append(temp_seq)
  
  return all_data

In [0]:
def indice(nb_salades,data):
  flat_list = [item for sublist in data[:nb_salades] for item in sublist]
  return len(flat_list)+1

In [0]:
def main_concat(limite_ecarts,limite_freq,func,dataset):
  base = func(dataset)
  results = HCDA(dataset,limite_freq,limite_ecarts,base)
  print_HCDA(results)
   
  return results

In [0]:
def main(limite_ecarts,limite_freq,func,path):
  all_results = []
  for name in get_all_files(path):
    base = func(sequence(name))
    results = HCDA(sequence(name),limite_freq,limite_ecarts,base)
    if results != []:
      all_results.append(results)
      print("DATASET : {}".format(name))
      print_HCDA(results)
      print("\n")

  return all_results